# PhysicsList


## 方法一：使用现有的 physics list

+ basic/B2
+ 只需要在主程序中调用对应的 physics list 即可，具体代码如下所示：
  ```
   G4VModularPhysicsList* physicsList = new FTFP_BERT;
   physicsList->RegisterPhysics(new G4StepLimiterPhysics());
   runManager->SetUserInitialization(physicsList);

  ```




## 方法二：继承 G4VModularPhysicsList 类

+ basic/B3
+ 需要继承 G4VModularPhysicsList 类，在 B3 中，该类为 PhysicsList。在该方法中，不需要自己额外写具体的物理过程，直接通过 RegisterPhysics() 函数注册即可。在 .cc 程序中，具体代码如下所示：
  ```
  //
   PhysicsList::PhysicsList()
  {
    SetVerboseLevel(1);
 
    // Default physics
    RegisterPhysics(new G4DecayPhysics());
 
    // EM physics
    RegisterPhysics(new G4EmStandardPhysics());
 
    // Radioactive decay
    RegisterPhysics(new G4RadioactiveDecayPhysics());
  }
 
  //
 
  void PhysicsList::SetCuts()
  {
    G4VUserPhysicsList::SetCuts();
  }

  ```
+ 在主程序中，需要如下代码进行调用：
  ```
  runManager->SetUserInitialization(new B3::PhysicsList);
  ```

## 方法三：继承 G4VPhysicsConstructor 类
+ extended/electromagnetic/TestEm0
+ extended/electromagnetic/TestEm15
+ 需要有继承自 G4VPhysicsConstructor 的类，在 TestEm0 中，该类为 PhysListEmStandard。需要具体的实现 ConstructProcess() 方法，需要针对不同的粒子，设置不同的物理过程：
  ```
  if (particleName == "gamma") {

       ph->RegisterProcess(new G4RayleighScattering, particle);
       ph->RegisterProcess(new G4PhotoElectricEffect, particle);
       G4ComptonScattering* cs   = new G4ComptonScattering;
       cs->SetEmModel(new G4KleinNishinaModel());
       ph->RegisterProcess(cs, particle);
       ph->RegisterProcess(new G4GammaConversion, particle);
  } 
  ```
+ 在 TestEm15 中，通过 AddProcess() 方法设置针对不同粒子的物理过程：
  ```
   if (particleName == "gamma") {
   // gamma
   pmanager->AddDiscreteProcess(new G4PhotoElectricEffect);
   pmanager->AddDiscreteProcess(new G4ComptonScattering);
   pmanager->AddDiscreteProcess(new G4GammaConversion);
   }
  ```
+ 同时还需要继承自 G4VModularPhysicsList 的类，在 TestEm0 中，该类为 PhysicsList。需要在该类中调用 ConstructProcess() 方法。

## 方法四：继承 G4VProcess 类
+ extended/exoticphysics/monopole
+ 需要自行构建物理过程，继承自 G4VProcess 类，在 monopole 中，该类为 G4MonopoleTransportation。
+ 其余部分与方法二和方法三类似。

+ 在 G4 中，一个反应需要 GetPhysicalInteractionLength(GPIL) 以及 DoIt 函数，其中 GPIL 函数根据反应截面的信息给出 step length，在该 step 结束后会调用 DoIt 函数，该函数对 particle 的能量，动量，方向，位置做了修改，以及生成次级的 track。

### 三种 GPIL 以及 DoIt 函数
+ AlongStepDoIt。该函数在粒子发生每一步 step 时被调用。对于实际情况，多个 G4VProcess 对应着多个 AlongStepDoIt 函数，每个 AlongStepDoIt 函数都会被调用，并更新 particle。在所有的调用完成后，会将更新的 particle 信息给到 track 中。
+ PostStepDoIt。如果某个 G4VProcess 有最小的 step length 或者被强制执行。